<a href="https://colab.research.google.com/github/anmolkhurana490/Sign-Language-Translator/blob/main/model_training/Gloss_To_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [7]:
# from transformers import pipeline 
from openai import OpenAI

In [8]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key='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',
)

In [9]:
# from google.colab import userdata
# hf_token = userdata.get('hf_token')

In [10]:
# from huggingface_hub import login
# login(hf_token)

In [11]:
# generator = pipeline(task='text-generation', model="meta-llama/Llama-3.2-3B-Instruct")

In [21]:
def generator(prompt, max_new_tokens=50):
    response = client.chat.completions.create(
      model="meta-llama/Llama-3.2-3B-Instruct",
      messages = [
        {
          "role": "user",
          "content": prompt
        }
      ],
      max_tokens=max_new_tokens,
      temperature=0.7,
    )

    return response.choices[0].message.content

In [22]:
gloss_input = "i/my/mine/me love/like books"
prompt = f'Convert tokens into a natural English sentence.\nGloss: {gloss_input}\nSentence:'

output = generator(prompt, max_new_tokens=50)
print(output)

I love reading books.


In [23]:
if prompt in output:
    print(output.replace(prompt, '').strip())
else:
    print(output.strip())

I love reading books.


In [25]:
def generate_text(gloss_input, last_text=''):
    instruct = 'You are a gloss-to-English converter. Output only the sentence using only the gloss token. No need to complete it with additional words. No explanations.'
    prompt = f'{instruct}\nGloss: {gloss_input}\nSentence:'

    max_tokens = len(gloss_input.split()) + len(prompt.split())

    output = generator(prompt, max_new_tokens=max_tokens)

    if prompt in output:
        output = output.replace(prompt, '').strip()
    if '(' in output:
        output = output.split('(')[0].strip()
    else:
        output = output.strip()

    output = output.replace('_', ' ').strip()
    return output.split('\n')[0]

In [26]:
gloss_input = "me love books"
generate_text(gloss_input)

'I love reading'

In [27]:
gloss_input = "he would be coming today COLLEGE TRAIN"
generate_text(gloss_input)

'he would be coming today'

In [28]:
gloss_input = "HELLO HOW YOU/YOUR FEEL TODAY"
generate_text(gloss_input)

'HELLO HOW ARE YOU'

In [29]:
input_texts = """HELLO HOW YOU/YOUR FEEL TODAY
I/ME THINK FUTURE CAREER PLAN
YOU/YOUR LIKE/LOVE BOOK_READ OR MOVIE/FILM
FRIEND/COMPANION INVITE PARTY WEEKEND
I/ME/MINE/MY WANT/NEED TRAVEL VISIT NEW CITY
YESTERDAY RAIN WEATHER BAD STILL I/ME/MINE/MY GO SCHOOL
TOMORROW I/ME PLAN STUDY EXAM PREPARE
FAMILY SUPPORT/HELP ME DECISION CAREER
YOU/YOUR CAN PLAY GAME SPORTS WITH FRIEND/COMPANION
TIME NOW GO SLEEP BECAUSE TOMORROW EARLY WAKE"""

for sentence in input_texts.split('\n'):
    print(sentence + ' -> ' + generate_text(sentence))

HELLO HOW YOU/YOUR FEEL TODAY -> HELLO HOW YOU/YOUR FEEL TODAY
I/ME THINK FUTURE CAREER PLAN -> I/ME THINK FUTURE CAREER PLAN
YOU/YOUR LIKE/LOVE BOOK_READ OR MOVIE/FILM -> You like your book or movie/film.
FRIEND/COMPANION INVITE PARTY WEEKEND -> FRIEND/COMPANION INVITE PARTY WEEKEND
I/ME/MINE/MY WANT/NEED TRAVEL VISIT NEW CITY -> I WANT TO VISIT NEW CITY
YESTERDAY RAIN WEATHER BAD STILL I/ME/MINE/MY GO SCHOOL -> YESTERDAY RAIN WEATHER BAD STILL I GO SCHOOL
TOMORROW I/ME PLAN STUDY EXAM PREPARE -> I plan to study for my exam tomorrow
FAMILY SUPPORT/HELP ME DECISION CAREER -> I need family support to help me make a decision about my career.
YOU/YOUR CAN PLAY GAME SPORTS WITH FRIEND/COMPANION -> YOU CAN PLAY GAME SPORTS WITH FRIEND/COMPANION
TIME NOW GO SLEEP BECAUSE TOMORROW EARLY WAKE -> TIME NOW GO SLEEP BECAUSE TOMORROW EARLY WAKE


Continuous Gloss to Text Generation

In [30]:
import time
from collections import deque

In [31]:
WINDOW_SIZE = 8
MIN_TRIGGER = 4
MAX_CONSUME = 6
SILENCE_TIMEOUT = 3
CONF_THRESHOLD = 0.7

In [32]:
# Buffer Manager Class
class GlossBuffer:
  def __init__(self):
    self.buffer = deque(maxlen=WINDOW_SIZE)
    self.last_gloss_time = 0

  def append_gloss(self, gloss):
    curr_time = time.time()
    self.update_buffer(curr_time)

    if gloss not in self.get_buffer():
      self.buffer.append((gloss, curr_time))

    self.last_gloss_time = curr_time

  def update_buffer(self, curr_time):
    curr_time = time.time()
    if curr_time - self.last_gloss_time > SILENCE_TIMEOUT:
      self.buffer.clear()

    while len(self.buffer) > 0 and self.buffer[0][1] < curr_time - SILENCE_TIMEOUT:
      self.buffer.popleft()

  def get_buffer(self):
    return [t for t,c in self.buffer]

  def get_gloss_text(self):
    gloss_list = self.get_buffer()

    if len(gloss_list) < MIN_TRIGGER:
      return ''

    # removing older glosses
    if len(gloss_list) > MAX_CONSUME:
      gloss_list = gloss_list[:MAX_CONSUME]

    return ' '.join(gloss_list)

In [33]:
gloss_buffer = GlossBuffer()

In [34]:
text_buffer = deque(maxlen=WINDOW_SIZE//2)

In [35]:
text = "HELLO_HI I_ME NAME ANMOL NICE MEET YOU HOW ARE YOU TODAY I_ME FEEL HAPPY BECAUSE WEATHER GOOD I_ME LIKE LOVE BOOK DO YOU LOVE ME READ I_ME PLAN FUTURE CAREER ENGINEER COMPUTER WHAT PLAN CAREER YOU YOU WANT STUDY WHERE UNIVERSITY DELHI OR MUMBAI I_ME THINK HARD WORK IMPORTANT SOMETIMES I_ME FEEL STRESS BUT FRIEND HELP THANK YOU SUPPORT NEXT WEEK I_ME TRAVEL DELHI TRAIN TICKET BOOK ALREADY I_ME LOVE FAMILY PARENTS SUPPORT WHAT TIME MEETING START PLEASE WAIT LITTLE SORRY I_ME LATE AFTER MEETING I_ME WANT MOVIE WATCH YOU LIKE MOVIE TOO WHICH MOVIE BEST FRIDAY NIGHT PLAN GOOD SEE YOU TOMORROW BYE"

In [36]:
import random

def generate_continue_text():
  # gloss prediction simulation from given text
  for gloss_input in text.split():
    time.sleep(0.5)

    gloss_input = gloss_input.split('_')[0]
    gloss_buffer.append_gloss(gloss_input)

    if random.random() < CONF_THRESHOLD:
      continue

    gloss_text = gloss_buffer.get_gloss_text()

    if gloss_text != '':
      text_list = list(text_buffer)
      gen_text = generate_text(gloss_text, ' '.join(text_list))
      text_buffer.extend(gen_text.split())
      print(gen_text)

In [37]:
generate_continue_text()

I name Anmol nice meet you
HOW
We are feeling happy because the weather is good.
Do you love books?
I plan my future
What is your career as a computer engineer?
MUMBAI I THINK HARD WORK IMPORTANT
SOMETIMES I FEEL STRESS
I feel stress
I appreciate your assistance.
FRIEND HELP THANK YOU SUPPORT
NEXT WEEK I TRAVEL DELHI
I travel to Delhi by train
I already love this ticket book.
Familien Eltern unterstützen
I was late after the meeting.
Which movie do you like better?


In [38]:
gloss_input = "YESTERDAY RAIN WEATHER BAD STILL I/ME/MINE/MY GO SCHOOL"

# Measuring Time taken
start_time = time.time()
print(generate_text(gloss_input))
print(f"Time taken: {time.time() - start_time} seconds")

I went to school yesterday despite the bad weather.
Time taken: 2.2235300540924072 seconds
